短语挖掘方法介绍

第一大类：Unigram topic modeling 一元的主题模型，主要包括：

1. Latent Semantic Analysis (LSA)
2. Probablistic Latent Semantic Analysis （pLSA）
3. Latent Dirichlet Allocation (LDA)

下面开始对这几种模型的介绍

### 1.  LSA
参考博文[关于 LSA（Latent Semantic Analysis）主题模型的个人理解](http://blog.csdn.net/cang_sheng_ta_ge/article/details/46708515)，另外一篇英文文章[Latent Semantic Analysis (LSA) for Text Classification Tutorial](http://mccormickml.com/2016/03/25/lsa-for-text-classification-tutorial/)

其主要思想感觉有点类似于词嵌入，我们假设有好多词，那可以将文档用词来进行one-hot encoding，但是这样会造成文章大量的稀疏表示，于是我们想法就是怎么能够将稀疏矩阵变为稠密矩阵，一个做法就是矩阵分解。

下面代码参考 https://github.com/chrisjmccormick/LSA_Classification

In [2]:
import pickle

In [38]:
raw_text_dataset = pickle.load( open( "../data/LSA_Classification/data/raw_text_dataset.pickle", "rb" ) )
X_train_raw = raw_text_dataset[0]
y_train_labels = raw_text_dataset[1] 
X_test_raw = raw_text_dataset[2]
y_test_labels = raw_text_dataset[3]

此处的数据使用的是路透社的数据，大概类别有100左右

In [39]:
print(len(X_train_raw),len(X_test_raw))

4743 4858


In [40]:
# The Reuters dataset consists of ~100 categories. However, we are going to
# simplify this to a binary classification problem. The 'positive class' will
# be the articles related to "acquisitions" (or "acq" in the dataset). All
# other articles will be negative.
y_train = ["acq" in y for y in y_train_labels]
y_test = ["acq" in y for y in y_test_labels]

print("  %d training examples (%d positive)" % (len(y_train), sum(y_train)))
print("  %d test examples (%d positive)" % (len(y_test), sum(y_test)))

  4743 training examples (590 positive)
  4858 test examples (620 positive)


计算 LSA 第一步就是就按tf-idf值，我们直接使用 sklearn 中的模块

In [100]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [101]:
# 我们去除了停用词
# 过滤掉出现文档超过50%的词
# 过滤掉最小出现2次的词
# 挑选出前1w个词
vectorizer = TfidfVectorizer(max_df=0.5, max_features=10000,
                             min_df=2, stop_words='english',
                             use_idf=True)

In [12]:
X_train_tfidf = vectorizer.fit_transform(X_train_raw)
print("  Actual number of tfidf features: %d" % X_train_tfidf.get_shape()[1])

  Actual number of tfidf features: 10000


In [13]:
X_train_tfidf.get_shape()

(4743, 10000)

In [14]:
from sklearn.decomposition import TruncatedSVD

下一步我们要做的就是对高维的特征矩阵进行分解，SVD奇异值分解。

X_train_tfidf 中每一行代表一篇文章，每一列代表一个词，每个数值都是tf-idf值。

奇异值分解做的事情就是将高纬度稀疏矩阵分解为3个矩阵连乘。网上找到一篇非常好的文章：[奇异值分解 (SVD) --- 几何意义](http://blog.sciencenet.cn/blog-696950-699432.html)

In [15]:
svd = TruncatedSVD(100)

In [17]:
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline

我们先对训练数据求取svd，然后对齐进行正则化，得到单位向量

In [18]:
lsa = make_pipeline(svd, Normalizer(copy=False))

得到转换后的特征值，并且这些特征占总的信息量为27%

In [19]:
X_train_lsa = lsa.fit_transform(X_train_tfidf)

In [20]:
explained_variance = svd.explained_variance_ratio_.sum()
print("  Explained variance of the SVD step: {}%".format(int(explained_variance * 100)))

  Explained variance of the SVD step: 27%


In [31]:
# Now apply the transformations to the test data as well.
X_test_tfidf = vectorizer.transform(X_test_raw)
X_test_lsa = lsa.transform(X_test_tfidf)

下面开始对文章进行分类

In [37]:
from sklearn.neighbors import KNeighborsClassifier
import time

In [39]:
print("\nClassifying tfidf vectors...")

# Time this step.
t0 = time.time()

# Build a k-NN classifier. Use k = 5 (majority wins), the cosine distance, 
# and brute-force calculation of distances.
knn_tfidf = KNeighborsClassifier(n_neighbors=5, algorithm='brute', metric='cosine')
knn_tfidf.fit(X_train_tfidf, y_train)

# Classify the test vectors.
p = knn_tfidf.predict(X_test_tfidf)

# Measure accuracy
numRight = 0;
for i in range(0,len(p)):
    if p[i] == y_test[i]:
        numRight += 1

print("  (%d / %d) correct - %.2f%%" % (numRight, len(y_test), float(numRight) / float(len(y_test)) * 100.0))
# Calculate the elapsed time (in seconds)
elapsed = (time.time() - t0)
print("  done in %.3fsec" % elapsed)


Classifying tfidf vectors...
  (4471 / 4858) correct - 92.03%
  done in 1.404sec


In [40]:
# Time this step.
t0 = time.time()

# Build a k-NN classifier. Use k = 5 (majority wins), the cosine distance, 
# and brute-force calculation of distances.
knn_lsa = KNeighborsClassifier(n_neighbors=5, algorithm='brute', metric='cosine')
knn_lsa.fit(X_train_lsa, y_train)

# Classify the test vectors.
p = knn_lsa.predict(X_test_lsa)

# Measure accuracy
numRight = 0;
for i in range(0,len(p)):
    if p[i] == y_test[i]:
        numRight += 1

print("  (%d / %d) correct - %.2f%%" % (numRight, len(y_test), float(numRight) / float(len(y_test)) * 100.0))

# Calculate the elapsed time (in seconds)
elapsed = (time.time() - t0)    
print("    done in %.3fsec" % elapsed)

  (4561 / 4858) correct - 93.89%
    done in 0.749sec


#### gensim使用
使用 gensim 来做 LSA 的分析，Latent Semantic Indexing (also known as Latent Semantic Analysis)，在 gensim 中对应的模块是models.lsimodel

In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [9]:
import pickle
raw_text_dataset = pickle.load( open( "../data/LSA_Classification/data/raw_text_dataset.pickle", "rb" ) )
X_train_raw = raw_text_dataset[0]
y_train_labels = raw_text_dataset[1] 
X_test_raw = raw_text_dataset[2]
y_test_labels = raw_text_dataset[3]

整个处理过程可以分为4步：

1. 将文档分词
2. 单词Lemmatize
3. idf值


In [10]:
# Tokenize the documents.

from nltk.tokenize import RegexpTokenizer

def docTokenizer(docs):
    # Split the documents into tokens.
    tokenizer = RegexpTokenizer(r'\w+')
    newdocs = []
    for idx in range(len(docs)):
        doc = docs[idx].lower()  # Convert to lowercase.
        newdocs.append(tokenizer.tokenize(doc))  # Split into words.

    # Remove numbers, but not words that contain numbers.
    newdocs = [[token for token in doc if not token.isnumeric()] for doc in newdocs]

    # Remove words that are only one character.
    newdocs = [[token for token in doc if len(token) > 1] for doc in newdocs]
    
    return newdocs

In [11]:
x_train_tokens = docTokenizer(X_train_raw)
x_test_tokens = docTokenizer(X_test_raw)

In [16]:
from nltk.stem.wordnet import WordNetLemmatizer
# Lemmatize all words in documents.
lemmatizer = WordNetLemmatizer()
x_train_tokens = [[lemmatizer.lemmatize(token) for token in doc] for doc in x_train_tokens]
x_test_tokens = [[lemmatizer.lemmatize(token) for token in doc] for doc in x_test_tokens]

下一步我们要做的就是建立corups了，去除超过在一半文档都出现的词，去除出现文档数小于20的词

In [18]:
# Remove rare and common tokens.
import numpy as np
from gensim.corpora import Dictionary


# Create a dictionary representation of the documents.
dictionary = Dictionary(np.concatenate((x_train_tokens,x_test_tokens)))

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.5)

2017-09-30 16:24:20,453 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2017-09-30 16:24:22,217 : INFO : built Dictionary(27466 unique tokens: ['bahia', 'cocoa', 'review', 'shower', 'continued']...) from 9601 documents (total 1214137 corpus positions)
2017-09-30 16:24:22,279 : INFO : discarding 24027 tokens: [('bahia', 1), ('shower', 6), ('the', 7371), ('in', 6381), ('alleviating', 2), ('and', 6869), ('for', 5439), ('temporao', 2), ('humidity', 3), ('restored', 18)]...
2017-09-30 16:24:22,281 : INFO : keeping 3439 tokens which were in no less than 20 and no more than 4800 (=50.0%) documents
2017-09-30 16:24:22,293 : INFO : resulting dictionary: Dictionary(3439 unique tokens: ['cocoa', 'review', 'continued', 'throughout', 'week']...)


此处去除完后，字典就剩 3439 了，下面我们处理语料

In [30]:
from gensim.models import TfidfModel
# Vectorize data.
tfidf = TfidfModel(dictionary=dictionary)
# Bag-of-words representation of the documents.
corups = [dictionary.doc2bow(doc) for doc in np.concatenate((x_train_tokens,x_test_tokens))]

In [31]:
corpus_tfidf = tfidf[corups]

In [26]:
from gensim.models import LsiModel,TfidfModel

In [33]:
lsi = LsiModel(corpus_tfidf, num_topics=100,id2word=dictionary)

2017-09-30 16:42:37,165 : INFO : using serial LSI version on this node
2017-09-30 16:42:37,167 : INFO : updating model with new documents
2017-09-30 16:42:38,488 : INFO : preparing a new chunk of documents
2017-09-30 16:42:38,674 : INFO : using 100 extra samples and 2 power iterations
2017-09-30 16:42:38,676 : INFO : 1st phase: constructing (3439, 200) action matrix
2017-09-30 16:42:38,922 : INFO : orthonormalizing (3439, 200) action matrix
2017-09-30 16:42:39,723 : INFO : 2nd phase: running dense svd on (200, 9601) matrix
2017-09-30 16:42:39,900 : INFO : computing the final decomposition
2017-09-30 16:42:39,903 : INFO : keeping 100 factors (discarding 20.809% of energy spectrum)
2017-09-30 16:42:39,909 : INFO : processed documents up to #9601
2017-09-30 16:42:39,912 : INFO : topic #0(24.459): -0.640*"v" + -0.364*"ct" + -0.289*"net" + -0.261*"loss" + -0.228*"shr" + -0.196*"mln" + -0.173*"rev" + -0.155*"profit" + -0.142*"qtr" + -0.098*"oper"
2017-09-30 16:42:39,914 : INFO : topic #1(16.

In [52]:
vectors = [[item[1] for item in vec] for vec in  lsi[corpus_tfidf]]

In [53]:
print(len(X_train_raw),len(X_test_raw),len(vectors))

4743 4858 9601


In [54]:
np.shape(vectors)

(9601, 100)

In [56]:
from sklearn.neighbors import KNeighborsClassifier
import time

print("\nClassifying tfidf vectors...")

# Time this step.
t0 = time.time()

# Build a k-NN classifier. Use k = 5 (majority wins), the cosine distance, 
# and brute-force calculation of distances.
knn_tfidf = KNeighborsClassifier(n_neighbors=5, algorithm='brute', metric='cosine')
knn_tfidf.fit(vectors[:4743], y_train)

# Classify the test vectors.
p = knn_tfidf.predict(vectors[4743:])

# Measure accuracy
numRight = 0;
for i in range(0,len(p)):
    if p[i] == y_test[i]:
        numRight += 1

print("  (%d / %d) correct - %.2f%%" % (numRight, len(y_test), float(numRight) / float(len(y_test)) * 100.0))
# Calculate the elapsed time (in seconds)
elapsed = (time.time() - t0)
print("  done in %.3fsec" % elapsed)


Classifying tfidf vectors...
  (4593 / 4858) correct - 94.55%
  done in 0.699sec


通过上面的预处理，我们模型的预测结果又有了提升 😂

#### LSA 的延伸
看到上面 LSA 方法的时候，其实我就想到了 word embeding，那我们能够用神经网络来做LSA嘛，思路：我们将document进行embeding，然后将词也进行embedding，然后经过一个3层全连接，输出词在document中出现的次数，我们不断的拟合上面的网络，就能得到 document 和 word 的向量了。

In [43]:
print(len(X_train_raw)+len(X_test_raw))

9601


In [96]:
import keras
import numpy as np

In [166]:
wordInput = keras.Input((1,),name='word_input',dtype='int32')
docInput = keras.Input((1,),name='doc_input',dtype='int32')
# countInput = keras.Input((1,),name='count_input',dtype='int32')

In [167]:
latent_topic_num = 100
wordEmbed = keras.layers.embeddings.Embedding(10000,latent_topic_num,input_length=1,
                                              embeddings_initializer=keras.initializers.VarianceScaling(2),name='word_embed')
docEmbed = keras.layers.embeddings.Embedding(9601,latent_topic_num,input_length=1,
                                  embeddings_initializer=keras.initializers.VarianceScaling(2),name='doc_embed')
word_embed = keras.layers.Reshape((latent_topic_num,))(wordEmbed(wordInput)) # can also use Flatten
doc_embed = keras.layers.Reshape((latent_topic_num,))(docEmbed(docInput))

In [168]:
(word_embed)

<tf.Tensor 'reshape_16/Reshape:0' shape=(?, 100) dtype=float32>

In [169]:
input_vector = keras.layers.concatenate([word_embed,doc_embed])

In [170]:
input_vector

<tf.Tensor 'concatenate_9/concat:0' shape=(?, 200) dtype=float32>

In [171]:
layer_len = 3
# layer_num = 10
layer_nums = [20,10,5]
for i in range(len(layer_nums)):
    input_vector = keras.layers.Dense(layer_nums[i],
                                      kernel_initializer=keras.initializers.VarianceScaling(2),
                                     activation='relu')(input_vector)
    
y_predict = keras.layers.Dense(1,kernel_initializer=keras.initializers.VarianceScaling(2))(input_vector)

In [172]:
model = keras.models.Model([wordInput,docInput],y_predict)
model.compile(optimizer='adam',loss='mse')

下面根据上面的模型，我们来定制我们的输入

In [103]:
from sklearn.feature_extraction.text import CountVectorizer

In [104]:
dnn_vectorizer = CountVectorizer(max_df=0.5, max_features=10000,
                             min_df=2, stop_words='english')

In [110]:
raw_documents = np.concatenate((X_train_raw,X_test_raw))

In [112]:
doc_tf = dnn_vectorizer.fit_transform(raw_documents)

In [113]:
doc_tf.get_shape()

(9601, 10000)

In [115]:
term_frequence = doc_tf.toarray()

In [116]:
wordInputs = []
docInputs = []
targetCounts = []
for i in range(len(term_frequence)):
    doc_frequence = term_frequence[i]
    for j in range(len(doc_frequence)):
        if doc_frequence[j] != 0:
            docInputs.append(i)
            wordInputs.append(j)
            targetCounts.append(doc_frequence[j])

In [139]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
word_input (InputLayer)          (None, 1)             0                                            
____________________________________________________________________________________________________
doc_input (InputLayer)           (None, 1)             0                                            
____________________________________________________________________________________________________
word_embed (Embedding)           (None, 1, 5)          48005       word_input[0][0]                 
____________________________________________________________________________________________________
doc_embed (Embedding)            (None, 1, 5)          50000       doc_input[0][0]                  
___________________________________________________________________________________________

In [140]:
len(wordInputs)

498923

In [175]:
model.fit([np.array(wordInputs),np.array(docInputs)],np.array(targetCounts),batch_size=10000,epochs=100)

Epoch 1/100
498923/498923 [==============================] - 2s - loss: 0.3101     
Epoch 2/100
498923/498923 [==============================] - 2s - loss: 0.3077     
Epoch 3/100
498923/498923 [==============================] - 2s - loss: 0.3062     
Epoch 4/100
498923/498923 [==============================] - 2s - loss: 0.3030     
Epoch 5/100
498923/498923 [==============================] - 2s - loss: 0.3010     
Epoch 6/100
498923/498923 [==============================] - 2s - loss: 0.2990     
Epoch 7/100
498923/498923 [==============================] - 2s - loss: 0.2963     
Epoch 8/100
498923/498923 [==============================] - 2s - loss: 0.2961     
Epoch 9/100
498923/498923 [==============================] - 2s - loss: 0.2949     
Epoch 10/100
498923/498923 [==============================] - 2s - loss: 0.2915     
Epoch 11/100
498923/498923 [==============================] - 2s - loss: 0.2895     
Epoch 12/100
498923/498923 [==============================] - 2s - loss: 0

In [155]:
np.shape(model.get_weights()[0])

(10000, 5)

In [156]:
np.shape(model.get_weights()[1])

(9601, 5)

进行分类

In [159]:
print(len(X_train_raw),len(X_test_raw))

4743 4858


In [176]:
#  (4183 / 4858) correct - 86.11%,
#  0.4882 - 85.78% 
# 100个hidden，100次，loss:0.3117 , correct - 86.39%
# 100个hidden，100次，loss:0.2095 , correct - 86.21%
# 可以说上面这种做法
# Time this step.
t0 = time.time()

# Build a k-NN classifier. Use k = 5 (majority wins), the cosine distance, 
# and brute-force calculation of distances.
knn_lsa = KNeighborsClassifier(n_neighbors=5, algorithm='brute', metric='cosine')
knn_lsa.fit(model.get_weights()[1][:4743], y_train)

# Classify the test vectors.
p = knn_lsa.predict(model.get_weights()[1][4743:])

# Measure accuracy
numRight = 0;
for i in range(0,len(p)):
    if p[i] == y_test[i]:
        numRight += 1

print("  (%d / %d) correct - %.2f%%" % (numRight, len(y_test), float(numRight) / float(len(y_test)) * 100.0))

# Calculate the elapsed time (in seconds)
elapsed = (time.time() - t0)    
print("    done in %.3fsec" % elapsed)

  (4188 / 4858) correct - 86.21%
    done in 0.515sec


结论：通过上面的dnn方法，baseline的model效果并不是很好

### 2. pLSA
在LSA中，我们构建了term和document的共现矩阵，
![图片](http://bos.nj.bpc.baidu.com/v1/agroup/afed8760421ed8b2b3d003e9b14f7f56673cc4a3)
这个的核心假设是：使用词袋模型来表示文章（即使丢失了term的顺序）也能很好的代表文章信息，LSA 通过svg方法，将document映射到一个隐向量空间上去。

我们更进一步，考虑引入一个隐含变量$z_k$，来表示文档主题，形成简单的贝叶斯网络，
![](http://bos.nj.bpc.baidu.com/v1/agroup/f67618811933d48654161ae53fe5f2d836ace8d8)

非常好的文档可以参考：

[概率语言模型及其变形系列 (1)-PLSA 及 EM 算法](http://blog.csdn.net/yangliuy/article/details/8330640)

[自然语言处理之 PLSA](http://zhikaizhang.cn/2016/06/17/%E8%87%AA%E7%84%B6%E8%AF%AD%E8%A8%80%E5%A4%84%E7%90%86%E4%B9%8BPLSA/)

[主题模型之 pLSA](http://blog.jqian.net/post/plsa.html)

里面重要的点是EM的思想，即先假设参数$\theta$已知，然后求隐变量的后验概率，然后此时知道隐变量后，就可以和数据X一起组成完整的数据集，此时再来求极值。


![](http://bos.nj.bpc.baidu.com/v1/agroup/4a83a5cf5beaced4dbe535759365c1bab4b2862b)

下面是具体的算法实现，代码参考自：https://github.com/laserwave/PLSA

In [62]:
from numpy import zeros, int8, log
from pylab import random
import sys
import jieba
import re
import time
import codecs

代码中使用 $lamda[i,k]$ 表示参数 $p(z_k|d_i)$, 用 $theta[k,j]$ 表示参数 $p(w_j|z_k)$, 用 $p[i,j,k]$ 表示隐藏变量的后验概率 $p(z_k|d_i,w_j)$。

第一步，我们需要初始化参数 $p(z_k|d_i)$, $p(w_j|z_k)$

In [63]:
def initializeParameters():
    for i in range(0, N):
        normalization = sum(lamda[i, :])
        for k in range(0, K):
            lamda[i, k] /= normalization

    for k in range(0, K):
        normalization = sum(theta[k, :])
        for j in range(0, M):
            theta[k, j] /= normalization

第二步是对于文档数据的处理，我们需要得到统计数据，$n(d_i),n(d_i,w_j)$

In [68]:
# segmentation, stopwords filtering and document-word matrix generating
# [return]:
# N : number of documents
# M : length of dictionary
# word2id : a map mapping terms to their corresponding ids
# id2word : a map mapping ids to terms
# X : document-word matrix, N*M, each line is the number of terms that show up in the document
def preprocessing(datasetFilePath, stopwordsFilePath):
    
    # read the stopwords file
    file = codecs.open(stopwordsFilePath, 'r', 'utf-8')
    stopwords = [line.strip() for line in file] 
    file.close()
    
    # read the documents
    file = codecs.open(datasetFilePath, 'r', 'utf-8')
    documents = [document.strip() for document in file] 
    file.close()

    # number of documents
    N = len(documents)

    wordCounts = [];
    word2id = {}
    id2word = {}
    currentId = 0;
    # generate the word2id and id2word maps and count the number of times of words showing up in documents
    for document in documents:
        segList = jieba.cut(document)
        wordCount = {}
        for word in segList:
            word = word.lower().strip()
            if len(word) > 1 and not re.search('[0-9]', word) and word not in stopwords:               
                if word not in word2id.keys():
                    word2id[word] = currentId;
                    id2word[currentId] = word;
                    currentId += 1;
                if word in wordCount:
                    wordCount[word] += 1
                else:
                    wordCount[word] = 1
        wordCounts.append(wordCount);
    
    # length of dictionary
    M = len(word2id)  

    # generate the document-word matrix
    X = zeros([N, M], int8)
    for word in word2id.keys():
        j = word2id[word]
        for i in range(0, N):
            if word in wordCounts[i]:
                X[i, j] = wordCounts[i][word];    

    return N, M, word2id, id2word, X

In [83]:
# set the default params and read the params from cmd
datasetFilePath = '../data/PLSA/dataset3.txt'
stopwordsFilePath = '../data/PLSA/stopwords.dic'
K = 10    # number of topic
maxIteration = 30
threshold = 10.0
topicWordsNum = 10
docTopicDist = '../data/PLSA/docTopicDistribution.txt'
topicWordDist = '../data/PLSA/topicWordDistribution.txt'
dictionary = '../data/PLSA/dictionary.dic'
topicWords = '../data/PLSA/topics.txt'

In [84]:
# preprocessing
N, M, word2id, id2word, X = preprocessing(datasetFilePath, stopwordsFilePath)

In [85]:
print(X.shape)

(50, 5757)


In [86]:
# lamda[i, j] : p(zj|di)
lamda = random([N, K])

# theta[i, j] : p(wj|zi)
theta = random([K, M])

# p[i, j, k] : p(zk|di,wj)
p = zeros([N, M, K])

initializeParameters()

第三步是我们需要计算E，M

In [87]:
def EStep():
    for i in range(0, N):
        for j in range(0, M):
            denominator = 0;
            for k in range(0, K):
                p[i, j, k] = theta[k, j] * lamda[i, k];
                denominator += p[i, j, k];
            if denominator == 0:
                for k in range(0, K):
                    p[i, j, k] = 0;
            else:
                for k in range(0, K):
                    p[i, j, k] /= denominator;

In [88]:
def MStep():
    # update theta
    for k in range(0, K):
        denominator = 0
        for j in range(0, M):
            theta[k, j] = 0
            for i in range(0, N):
                theta[k, j] += X[i, j] * p[i, j, k]
            denominator += theta[k, j]
        if denominator == 0:
            for j in range(0, M):
                theta[k, j] = 1.0 / M
        else:
            for j in range(0, M):
                theta[k, j] /= denominator
        
    # update lamda
    for i in range(0, N):
        for k in range(0, K):
            lamda[i, k] = 0
            denominator = 0
            for j in range(0, M):
                lamda[i, k] += X[i, j] * p[i, j, k]
                denominator += X[i, j];
            if denominator == 0:
                lamda[i, k] = 1.0 / K
            else:
                lamda[i, k] /= denominator

接着我们定义最大似然函数

![](http://bos.nj.bpc.baidu.com/v1/agroup/36dda384ca41b649d5896500a3c6bdb63ecdcb64)
第一部分是定值，我们主要优化第二部分

In [89]:
# calculate the log likelihood
def LogLikelihood():
    loglikelihood = 0
    for i in range(0, N):
        for j in range(0, M):
            tmp = 0
            for k in range(0, K):
                tmp += theta[k, j] * lamda[i, k]
            if tmp > 0:
                loglikelihood += X[i, j] * log(tmp)
    return loglikelihood

In [90]:

# output the params of model and top words of topics to files
def output():
    # document-topic distribution
    file = codecs.open(docTopicDist,'w','utf-8')
    for i in range(0, N):
        tmp = ''
        for j in range(0, K):
            tmp += str(lamda[i, j]) + ' '
        file.write(tmp + '\n')
    file.close()
    
    # topic-word distribution
    file = codecs.open(topicWordDist,'w','utf-8')
    for i in range(0, K):
        tmp = ''
        for j in range(0, M):
            tmp += str(theta[i, j]) + ' '
        file.write(tmp + '\n')
    file.close()
    
    # dictionary
    file = codecs.open(dictionary,'w','utf-8')
    for i in range(0, M):
        file.write(id2word[i] + '\n')
    file.close()
    
    # top words of each topic
    file = codecs.open(topicWords,'w','utf-8')
    for i in range(0, K):
        topicword = []
        ids = theta[i, :].argsort()
        for j in ids:
            topicword.insert(0, id2word[j])
        tmp = ''
        for word in topicword[0:min(topicWordsNum, len(topicword))]:
            tmp += word + ' '
        file.write(tmp + '\n')
    file.close()

In [91]:
# EM algorithm
oldLoglikelihood = 1
newLoglikelihood = 1
for i in range(0, maxIteration):
    EStep()
    MStep()
    newLoglikelihood = LogLikelihood()
    print("[", time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())), "] ", i+1, " iteration  ", str(newLoglikelihood))
    if(oldLoglikelihood != 1 and newLoglikelihood - oldLoglikelihood < threshold):
        break
    oldLoglikelihood = newLoglikelihood

output()

[ 2017-10-11 15:34:36 ]  1  iteration   -119117.603574
[ 2017-10-11 15:35:15 ]  2  iteration   -116686.457897
[ 2017-10-11 15:35:55 ]  3  iteration   -113447.248935
[ 2017-10-11 15:36:34 ]  4  iteration   -110041.623584
[ 2017-10-11 15:37:14 ]  5  iteration   -107015.812203
[ 2017-10-11 15:37:53 ]  6  iteration   -104487.345965
[ 2017-10-11 15:38:33 ]  7  iteration   -102420.812678
[ 2017-10-11 15:39:12 ]  8  iteration   -100820.586452
[ 2017-10-11 15:39:52 ]  9  iteration   -99754.3630987
[ 2017-10-11 15:40:31 ]  10  iteration   -99132.5837025
[ 2017-10-11 15:41:10 ]  11  iteration   -98773.0945751
[ 2017-10-11 15:41:50 ]  12  iteration   -98544.0665881
[ 2017-10-11 15:42:33 ]  13  iteration   -98366.1570079
[ 2017-10-11 15:43:12 ]  14  iteration   -98181.4161962
[ 2017-10-11 15:43:52 ]  15  iteration   -97969.1043667
[ 2017-10-11 15:44:31 ]  16  iteration   -97753.8946986
[ 2017-10-11 15:45:10 ]  17  iteration   -97570.5593186
[ 2017-10-11 15:45:50 ]  18  iteration   -97442.3102903
[

In [92]:
!cat ../data/PLSA/topics.txt

孩子 红衣 潘先生 字条 交警 告诉 未成年人 性侵 网友 视频 
新新 男子 民警 孩子 学校 王某 男童 警官证 刘先生 酒吧 
周杰伦 记者 北京 书画院 微信 警方 公众 嫌疑人 民警 王超 
何天 司机 面包车 剑锋 小偷 公交车 老人 辛某 乘客 记者 
车辆 高速 飙车 事故 广州 发生 交警 分红 记者 行驶 
分红 村民 银行 民警 腾冲 婆婆 大墩 刘某 顺德 土豪 
孩子 家长 求婚 李女士 儿子 朋友 交警 教育 小苏 老师 
手机 景区 游客 龙池 救援 村民 搜救 胡敏 救援队 登山 
报警 民警 马赛克 警方 微博 嫌犯 拨打 治疗 照片 行政拘留 
刘尧 河源 假币 记者 警方 物流 女士 举报 带走 医院 


上面的服务使用起来特别的慢。。使用

下面我使用gensim中的plsa方法，在gensim中并没有专门实现plsa，只有LDA，通过设置LDA参数可以来实现plsa，所有我们先来看LDA的。

### 3. LDA

独立同分布 independent and identically distributed (i.i.d.)

In [97]:
# Set up log to external log file
# import logging
# logging.basicConfig(filename='lda_model.log', format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Set up log to terminal
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)